In [53]:
from collections import defaultdict
import itertools
import json
import os
import re
import warnings
import xml.etree.ElementTree as et
from collections import defaultdict
from tempfile import NamedTemporaryFile
from typing import Any, Dict, List, Optional, Tuple, cast

import numpy as np
from grobid_client.grobid_client import GrobidClient


from papermage.magelib import (
    BlocksFieldName,
    Box,
    Document,
    Entity,
    Metadata,
    PagesFieldName,
    RowsFieldName,
    Span,
    TokensFieldName,
)
from papermage.magelib.box import Box
from papermage.parsers.parser import Parser
from papermage.parsers.grobid_parser import GROBID_VILA_MAP
from papermage.utils.merge import cluster_and_merge_neighbor_spans

from papermage_components.reading_order_predictor import *

In [54]:
NS = {"tei": "http://www.tei-c.org/ns/1.0"}
grobid_server_url = "http://himalayan.lti.cs.cmu.edu:8070"
input_pdf_path = "data/AM_Creep_Papers/On the creep performance of the Ti‐6Al‐4V alloy processed by additive manufacturing.pdf"

In [4]:
grobid_config = {
            "grobid_server": grobid_server_url,
            "batch_size": 1000,
            "sleep_time": 5,
            "timeout": 60,
            "coordinates": sorted(
                set((*GROBID_VILA_MAP.keys(), "s", "ref", "body", "item", "persName"))
            ),
        }
with NamedTemporaryFile(mode="w", delete=False) as f:
    json.dump(grobid_config, f)
    config_path = f.name

client = GrobidClient(config_path=config_path, check_server=True)

GROBID server is up and running


In [19]:
(_, _, xml) = client.process_pdf(
            service="processFulltextDocument",
            pdf_file=input_pdf_path,
            generateIDs=False,
            consolidate_header=False,
            consolidate_citations=False,
            include_raw_citations=False,
            include_raw_affiliations=False,
            tei_coordinates=True,
            segment_sentences=True,
        )

xml_root = et.fromstring(xml)

In [22]:
page_dimensions = get_page_dimensions(xml_root)
section_to_boxes = get_coords_by_section(xml_root, page_dimensions)
consolidated_boxes = {
    section: segment_and_consolidate_boxes(section_boxes)
    for section, section_boxes in section_to_boxes.items()
}


In [ ]:
segment_and_consolidate_boxes(section_to_boxes[""])